# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235143065922                   -0.50    8.0         
  2   -7.250325509980       -1.82       -1.40    1.0   5.10ms
  3   -7.251160572555       -3.08       -2.18    1.0   5.13ms
  4   -7.251244181606       -4.08       -2.14    2.0   6.28ms
  5   -7.251333586641       -4.05       -2.71    1.0   5.17ms
  6   -7.251338200893       -5.34       -3.20    1.0   5.31ms
  7   -7.251338713929       -6.29       -3.56    2.0   6.17ms
  8   -7.251338790028       -7.12       -4.06    1.0   5.41ms
  9   -7.251338797387       -8.13       -4.45    2.0   6.43ms
 10   -7.251338798508       -8.95       -5.00    1.0   5.50ms
 11   -7.251338798681       -9.76       -5.31    3.0   7.63ms
 12   -7.251338798700      -10.71       -5.75    1.0   5.80ms
 13   -7.251338798703      -11.49       -6.04    2.0   6.75ms
 14   -7.251338798704      -12.03       -6.44    1.0   5.72ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08291925262594536
[ Info: Arnoldi iteration step 2: normres = 0.6218991656235237
[ Info: Arnoldi iteration step 3: normres = 0.6269751794678439
[ Info: Arnoldi iteration step 4: normres = 0.25088822966847946
[ Info: Arnoldi iteration step 5: normres = 0.4515680104144129
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.13e-02, 7.86e-02, 3.31e-01, 2.96e-01, 2.24e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3876558856742675
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.70e-03, 1.67e-01, 3.12e-01, 9.79e-02, 8.55e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09278249123010118
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.81e-04, 2.13e-02, 2.16e-02, 5.98e-02, 5.76e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12816270341811678
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.01e-